In [4]:
import pandas as pd
import glob
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import os
sns.set()

In [5]:
# Analyzes regression in app benchmark and prints relative differences in appRegression.csv

drop_first_rows_percent = 5
drop_last_rows_percent = 20

folder = "../../results_all/app_history_vm/"
filename = "../../results_aggr/app_history_vm.csv"

print(folder)

../../results_all/app_history_vm/


In [6]:
def drop_rows(df):
    df_length = df.shape[0]
    df = df.drop(df.tail(int(df_length * drop_last_rows_percent / 100)).index)
    df = df.drop(df.head(int(df_length * drop_first_rows_percent / 100)).index)
    return df

rows = []

all_runs = glob.glob(os.path.join(folder, "run[0-9]"))
for fullRunFolder in all_runs:
    all_numbers = glob.glob(os.path.join(fullRunFolder, "*"))
    for fullNumberFolder in all_numbers:

        all_files = glob.glob(os.path.join(fullNumberFolder, "*"))
        size = len(all_files)

        if size != 22:
            print(f"{size} elements in folder {fullNumberFolder}, skip.")
        else:
            tmp = fullNumberFolder.split(os.sep)

            #Parse run and commit number from path structure
            run = tmp[-2][-1]
            number = tmp[-1]
            print(f"Run {run} Number {number}")

            #Read latency file for both types (old and new)
            insertsOld1 = pd.read_csv(fullNumberFolder + "/latenciesInserts1Old.csv",
                             header=None,
                             delim_whitespace=True,
                             names=["batch","latency (ms)"])
            insertsOld1.sort_values(by=["batch"], inplace=True)

            insertsOld2 = pd.read_csv(fullNumberFolder + "/latenciesInserts2Old.csv",
                                        header=None,
                                        delim_whitespace=True,
                                        names=["batch","latency (ms)"])
            insertsOld2.sort_values(by=["batch"], inplace=True)
            insertsOld = pd.concat([drop_rows(insertsOld1), drop_rows(insertsOld2)])

            insertsNew1 = pd.read_csv(fullNumberFolder + "/latenciesInserts1New.csv",
                             header=None,
                             delim_whitespace=True,
                             names=["batch","latency (ms)"])
            insertsNew1.sort_values(by=["batch"], inplace=True)
            insertsNew2 = pd.read_csv(fullNumberFolder + "/latenciesInserts2New.csv",
                             header=None,
                             delim_whitespace=True,
                             names=["batch","latency (ms)"])
            insertsNew2.sort_values(by=["batch"], inplace=True)

            insertsNew = pd.concat([drop_rows(insertsNew1), drop_rows(insertsNew2)])



            queries1Old = pd.read_csv(fullNumberFolder + "/latenciesQueries1Old.csv",
                             header=None,
                             delim_whitespace=True,
                             names=["query","latency (ms)"])
            queries1Old.sort_values(by=["query"], inplace=True)
            queries1Old = drop_rows(queries1Old)
            

            queries2Old = pd.read_csv(fullNumberFolder + "/latenciesQueries2Old.csv",
                             header=None,
                             delim_whitespace=True,
                             names=["query","latency (ms)"])
            queries2Old.sort_values(by=["query"], inplace=True)

            queries2Old = drop_rows(queries2Old)

            queries1New = pd.read_csv(fullNumberFolder + "/latenciesQueries1New.csv",
                             header=None,
                             delim_whitespace=True,
                             names=["query","latency (ms)"])
            queries1New.sort_values(by=["query"], inplace=True)
            queries1New = drop_rows(queries1New)


            queries2New = pd.read_csv(fullNumberFolder + "/latenciesQueries2New.csv",
                             header=None,
                             delim_whitespace=True,
                             names=["query","latency (ms)"])
            queries2New.sort_values(by=["query"], inplace=True)
            queries2New = drop_rows(queries2New)

            #Find median latency for both types
            medianInsertsOld = insertsOld["latency (ms)"].median()
            medianInsertsNew = insertsNew["latency (ms)"].median()

            medianQueries1Old = queries1Old["latency (ms)"].median()
            medianQueries1New = queries1New["latency (ms)"].median()

            medianQueries2Old = queries2Old["latency (ms)"].median()
            medianQueries2New = queries2New["latency (ms)"].median()

            #Calculate diff
            diffInserts = ((medianInsertsNew / medianInsertsOld) - 1) * 100
            diffQueries1 = ((medianQueries1New / medianQueries1Old) - 1) * 100
            diffQueries2 = ((medianQueries2New / medianQueries2Old) - 1) * 100

            #Append to rows
            rows.append({
                "run" : run,
                "number" : int(number),
                "diffInserts": diffInserts,
                "diffQueries1": diffQueries1,
                "diffQueries2": diffQueries2
            })

Run 1 Number 1
Run 1 Number 10
Run 1 Number 11
Run 1 Number 12
Run 1 Number 13
Run 1 Number 14
Run 1 Number 15
Run 1 Number 16
Run 1 Number 17
Run 1 Number 18
Run 1 Number 19
Run 1 Number 2
Run 1 Number 20
Run 1 Number 21
Run 1 Number 22
Run 1 Number 23
Run 1 Number 24
Run 1 Number 25
Run 1 Number 26
Run 1 Number 27
Run 1 Number 28
Run 1 Number 29
Run 1 Number 3
Run 1 Number 30
Run 1 Number 31
Run 1 Number 32
Run 1 Number 33
Run 1 Number 34
Run 1 Number 35
Run 1 Number 36
Run 1 Number 37
Run 1 Number 38
Run 1 Number 39
Run 1 Number 4
Run 1 Number 40
Run 1 Number 41
Run 1 Number 42
Run 1 Number 43
Run 1 Number 44
Run 1 Number 45
Run 1 Number 46
Run 1 Number 47
Run 1 Number 48
Run 1 Number 49
Run 1 Number 5
Run 1 Number 50
Run 1 Number 51
Run 1 Number 52
Run 1 Number 53
Run 1 Number 54
Run 1 Number 55
Run 1 Number 56
Run 1 Number 57
Run 1 Number 58
Run 1 Number 59
Run 1 Number 6
Run 1 Number 60
Run 1 Number 61
Run 1 Number 62
Run 1 Number 63
Run 1 Number 64
Run 1 Number 65
Run 1 Number 6

In [7]:
# Convert to data frame
df_result = pd.DataFrame(rows)
df_result.sort_values(by=["number"], inplace=True)
df_result.describe()

,number,diffInserts,diffQueries1,diffQueries2
count,207.000000,207.000000,207.000000,207.000000
mean,35.000000,1.145823,-3.831194,-2.447790
std,19.964775,2.012838,2.451890,1.394437
min,1.000000,-3.032443,-6.947120,-5.294407
25%,18.000000,-0.101987,-5.100890,-3.334965
50%,35.000000,0.879360,-4.589847,-2.645360
75%,52.000000,2.086533,-3.900286,-2.013392
max,69.000000,8.329394,9.247988,1.858124


In [8]:
df_result.to_csv(filename)
